In [1]:
!pip install gradio langchain langchain_community arxiv pymupdf PyPDF2

In [2]:
!pip install transformers

In [3]:
!pip install gradio langchain transformers PyPDF2 torch

  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB 245.8 kB/s eta 0:13:47
   ---------------------------------------- 0.0/203.1 MB 245.8 kB/s eta 0:13:47
   ---------------------------------------- 0.1/203.1 MB 514.3 kB/s eta 0:06:35
   ---------------------------------------- 0.2/203.1 MB 583.1 kB/s eta 0:05:49
   ---------------------------------------- 0.3/203.1 MB 886.2 kB/s eta 0:03:49
   ---------------------------------------- 0.4/203.1 MB 969.8 kB/s eta 0:03:30
   ---------------------------------------- 0.5/203.1 MB 1.1 MB/s eta 0:03:07
   ---------------------------------------- 0.5/203.1 MB 1.1 MB/s eta 0:03:13
   ----------------

In [7]:
import gradio as gr
from langchain.text_splitter import CharacterTextSplitter
from PyPDF2 import PdfReader
from transformers import pipeline
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

def summarize_pdf(pdf_file_path, model_name):
    summarizer = pipeline('summarization', model=model_name, min_length=100, max_length=200, device=0 if torch.cuda.is_available() else -1)
    loader = PdfReader(pdf_file_path.name)  # Use `name` attribute for file path in Gradio
    text = ""
    for page in loader.pages:
        text += page.extract_text()
    
    summary = summarizer(text)
    return summary[0]['summary_text']  # Extract the summary text

def main():
    input_pdf_path = gr.File(label="Upload PDF")
    select_model = gr.Dropdown(
        choices=["allenai/led-large-16384-arxiv", "philschmid/bart-large-cnn-samsum", "csebuetnlp/mT5_multilingual_XLSum"],
        label="Select Model"
    )
    output_summary = gr.Textbox(label="Summary")

    iface = gr.Interface(
        fn=summarize_pdf,
        inputs=[input_pdf_path, select_model],
        outputs=[output_summary],
        title="PDF Summarizer",
        description="Upload a PDF file and select a model to get its summary.",
    )
    
    iface.launch(share=True, inbrowser=True)  # Use `inbrowser=True` for Jupyter

if __name__ == "__main__":
    main()


Using device: cpu
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://b4ae17ad98b5925ac8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
